In [3]:
import csv
import os
import pandas as pd

In [5]:
import geopandas as gpd
import json
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')
import os
gdfs = []
dfs= []

json_dir = '../../data'
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        filepath = os.path.join(json_dir, filename)
        response = open(filepath)
        data = json.load(response)
        df = pd.json_normalize(data["features"], max_level=10)
        gdf = gpd.read_file(filepath)
        gdf = gdf.to_crs('EPSG:4326')
        dfs.append(df)
        gdfs.append(gdf)

In [6]:
all_routes_gdf = gpd.GeoDataFrame(pd.concat(gdfs))
all_routes_df = pd.concat(dfs)

lon = 22.38
lat = 60.41
geometry = [Point(lon, lat)]
kaarina_gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
kaarina_gdf

,geometry
0,POINT (22.38000 60.41000)


In [7]:
kaarina_3067 = kaarina_gdf
routes_3067 = all_routes_gdf




routes_3067 = routes_3067.to_crs("EPSG:3067")
kaarina_3067 = kaarina_3067.to_crs("EPSG:3067")

# Kilometrin "buffer" alue havaintoasemalle
kaarina_3067["buffer1000"] = kaarina_3067.geometry.buffer(1000)
kaarina_3067["buffer500"] = kaarina_3067.geometry.buffer(500)
kaarina_3067["buffer100"] = kaarina_3067.geometry.buffer(100)



timestamps1000 = []
timestamps500 = []
timestamps100 = []



count1000 = 0
count500 = 0
count100 = 0

# routes_3067
# kaarina_3067
for route in routes_3067.geometry:
    if (route.intersects(kaarina_3067['buffer1000']) | route.within(kaarina_3067['buffer1000'])).any():
        count1000 += 1
        if (route.intersects(kaarina_3067['buffer500']) | route.within(kaarina_3067['buffer500'])).any():
            count500 += 1
            if (route.intersects(kaarina_3067['buffer100']) | route.within(kaarina_3067['buffer100'])).any():
                count100 += 1

kaarina_3067['Reitit(1000m)'] = count1000
kaarina_3067['Reitit(500m)'] = count500
kaarina_3067['Reitit(100m)'] = count100




In [8]:

kaarina_3067

,geometry,buffer1000,buffer500,buffer100,Reitit(1000m),Reitit(500m),Reitit(100m)
0,POINT (245643.727 6705998.062),"POLYGON ((246643.727 6705998.062, 246638.912 6...","POLYGON ((246143.727 6705998.062, 246141.319 6...","POLYGON ((245743.727 6705998.062, 245743.245 6...",232,134,19


In [9]:
list = os.listdir(json_dir)
print(f"{list[0].strip('.json')} ja {list[-2].strip('.json')} välillä mitatut alueella olevat reitit:")
print(f"1000 metrin säde: {kaarina_3067.iloc[0]['Reitit(1000m)']}")
print(f"500 metrin säde: {kaarina_3067.iloc[0]['Reitit(500m)']}")
print(f"100 metrin säde: {kaarina_3067.iloc[0]['Reitit(100m)']}")

2024-03-05 ja 2024-04-15 välillä mitatut alueella olevat reitit:
1000 metrin säde: 232
500 metrin säde: 134
1000 metrin säde: 19


In [10]:
import numpy as np
cols = routes_3067.columns
kaarina_routes = pd.DataFrame(columns=cols)

kaarina_routes["1000m"] = ""
kaarina_routes["500m"] = ""
kaarina_routes["100m"] = ""


for i in range(len(routes_3067)):
    if (routes_3067.iloc[i]["geometry"].intersects(kaarina_3067['buffer1000']) | routes_3067.iloc[i]["geometry"].within(kaarina_3067['buffer1000'])).any():
        kaarina_routes.loc[i] = routes_3067.iloc[i]
        kaarina_routes["1000m"].loc[i] = True
        if (routes_3067.iloc[i]["geometry"].intersects(kaarina_3067['buffer500']) | routes_3067.iloc[i]["geometry"].within(kaarina_3067['buffer500'])).any():
            kaarina_routes["500m"].loc[i] = True
            if (routes_3067.iloc[i]["geometry"].intersects(kaarina_3067['buffer100']) | routes_3067.iloc[i]["geometry"].within(kaarina_3067['buffer100'])).any():
                     kaarina_routes["100m"].loc[i] = True

kaarina_routes["500m"] = kaarina_routes["500m"].fillna(False)
kaarina_routes["100m"] = kaarina_routes["100m"].fillna(False)


In [11]:
kaarina_routes[["1000m","500m","100m"]].head(5)

,1000m,500m,100m
643,True,False,False
644,True,True,False
645,True,True,False
646,True,True,False
647,True,True,False


In [12]:
# Tarkastelu manuaalisesti
route1 = all_routes_df.loc[all_routes_df["properties.id"] == kaarina_routes["id"].iloc[0]]
route1["geometry.coordinates"].values[0]


[[22.387908, 60.39881, 0.0],
 [22.387685, 60.399865, 0.0],
 [22.387875, 60.402937, 0.0]]

In [13]:
# Kaarinan alueella olevien reittien tiedot
linestring_count = 0
point_count = 0
for i in range(len(kaarina_routes)):
    if kaarina_routes["geometry"].iloc[i].geom_type == "LineString":
        linestring_count += 1
    elif kaarina_routes["geometry"].iloc[i].geom_type == "Point":
        point_count += 1
    else:
        print("error")
print(f"Alueella olevan vektoridatan määrä: {linestring_count}")
print(f"Alueella olevan pistemäisen datan määrä: {point_count}")

Alueella olevan vektoridatan määrä: 227
Alueella olevan pistemäisen datan määrä: 5


In [36]:
#Kaarinan havainstoaseman 1000m alueella suoritettujen reittien ajankohdat
import datetime
kaarina_routes["endTime"] = pd.to_datetime(kaarina_routes["endTime"])
kaarina_routes["startTime"] = pd.to_datetime(kaarina_routes["startTime"])
duration_hours = (kaarina_routes["endTime"] - kaarina_routes["startTime"]).dt.total_seconds() / 3600

duration_by_day = duration_hours.groupby(kaarina_routes['endTime'].dt.to_period('D')).sum()
duration_by_day


endTime
2024-03-14    1.719722
2024-03-27    0.587778
2024-03-28    1.673056
2024-03-29    4.297222
2024-04-11    0.589444
Freq: D, dtype: float64

In [37]:
#Kaarinan havainstoaseman 500m alueella suoritettujen reittien ajankohdat
buffer500_df = kaarina_routes.loc[kaarina_routes["500m"] == True]
duration_hours = (buffer500_df["endTime"] - buffer500_df["startTime"]).dt.total_seconds() / 3600

duration_by_day = duration_hours.groupby(buffer500_df['endTime'].dt.to_period('D')).sum()
duration_by_day


endTime
2024-03-14    1.513611
2024-03-27    0.226389
2024-03-28    0.773889
2024-03-29    2.230556
2024-04-11    0.429167
Freq: D, dtype: float64

In [38]:
#Kaarinan havainstoaseman 100m alueella suoritettujen reittien ajankohdat
buffer100_df = kaarina_routes.loc[kaarina_routes["100m"] == True]
duration_hours = (buffer100_df["endTime"] - buffer100_df["startTime"]).dt.total_seconds() / 3600

duration_by_day = duration_hours.groupby(buffer100_df['endTime'].dt.to_period('D')).sum()
duration_by_day


endTime
2024-03-27    0.039444
2024-03-29    0.528611
Freq: D, dtype: float64

In [62]:
datetime.date.today()
date = datetime.date.today()
export = kaarina_routes
# kaarina_routes.to_csv(f"kaarina_routes_{date}.csv")
export.to_csv(f"kaarina_routes_{date}.csv", index=False)